In [1]:
#OpenCV module
import cv2
#os module for reading training data directories and paths
import os
#numpy to convert python lists to numpy arrays as it is needed by OpenCV face recognizers
import numpy as np

#matplotlib for display our images
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import Image

In [2]:

#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["","Person-1", "Person-2", "Person-3","Person-4","Person-5","Person-6", "Person-6", "Person-8","Person-9","Person-10","Person-11", "Person-12", "Person-13","Person-14","Person-15","Person-16", "Person-17", "Person-18","Person-19","Person-20","Person-21", "Person-22", "Person-23","Person-24","Person-25","Person-26", "Person-27", "Person-28"]

In [3]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    #img = cv2.imread('/Users/guneevkaur/Desktop/datasets/training-data/s1/1.png')

    #cv2.imshow("Shrinked image", img)
    #key = cv2.waitKey()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('/Users/guneevkaur/anaconda/pkgs/opencv-2.4.11-py27_1/share/OpenCV/lbpcascades/lbpcascade_frontalface.xml')
    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]


In [4]:
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    dirs.sort()
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        #print dir_name
        label = int(dir_name.replace("s", ""))
        #print label
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = os.path.join(data_folder_path, dir_name)
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        subject_images_names.sort()
        #print subject_images_names
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if not image_name.endswith(".png"):# and not image_name.endswith("2.png"):
                continue;
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = os.path.join(subject_dir_path, image_name)
            print image_path
            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            #cv2.imshow("Training on image...", cv2.resize(image, (400, 500)))
            #cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)
            
            max_height = 200
            max_width = 200
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                height, width = face.shape[:2]
                fy = max_height / float(height)
                fx = max_width / float(width)
                face= cv2.resize(face, None, fx=fx,
                                  fy=fy, interpolation=cv2.INTER_AREA)
                #add face to list of faces
                faces.append(face)
                #add label for this face
                print label
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [5]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data")
#print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s1/1.png
1
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s1/2.png
1
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s1/3.png
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s10/1.png
10
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s10/2.png
10
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s10/3.png
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s11/1.png
11
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s11/2.png
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s11/3.png
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s12/1.png
12
/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/training-data/s12/2.png
12
/Users/guneevkaur/Desktop/BIOMETRIC CODE/P

In [6]:
#create our LBPH face recognizer 
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#or use EigenFaceRecognizer by replacing above line with 
face_recognizer = cv2.face.EigenFaceRecognizer_create()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.FisherFaceRecognizer_create()


In [7]:
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))


In [8]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)


In [9]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    
    img = test_img.copy()
    
    #detect face from the image
    face, rect = detect_face(img)
    
    max_height = 200
    max_width = 200
    
    if face is None:
        return "0"
    height, width = face.shape[:2]

    fy = max_height / float(height)
    fx = max_width / float(width)
    face= cv2.resize(face, None, fx=fx, 
                 fy=fy, interpolation=cv2.INTER_AREA)

    #plt.imshow(face)
    #predict the image using our face recognizer 
    label, confidence = face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return label

In [10]:
test_data_folder_path = "/Users/guneevkaur/Desktop/BIOMETRIC CODE/Prediction_dataset/testing-data"
test_dirs = os.listdir(test_data_folder_path)
test_dirs.sort()
correct_pred = 0
test_subjects_count = 0

for directory in test_dirs:
    if directory == '.DS_Store':
        continue
    #print os.path.join(test_data_folder_path, directory, "1.png")
    subject_images_names = os.listdir(os.path.join(test_data_folder_path, directory))
    subject_images_names.sort()
    for img in subject_images_names:
        if not img.endswith("1.png"):
            continue
        test_img = cv2.imread(os.path.join(test_data_folder_path, directory, img))
        #print test_img
        test_label = predict(test_img)
        print directory.replace("s", "Person-"), "Person-{}".format(str(test_label))
        if directory.replace("s", "Person-") == "Person-{}".format(str(test_label)):
            correct_pred += 1
        test_subjects_count += 1

Person-1 Person-1
Person-10 Person-7
Person-11 Person-16
Person-12 Person-12
Person-13 Person-13
Person-14 Person-14
Person-15 Person-15
Person-16 Person-22
Person-17 Person-17
Person-18 Person-0
Person-19 Person-19
Person-2 Person-2
Person-20 Person-20
Person-21 Person-21
Person-22 Person-22
Person-23 Person-22
Person-24 Person-19
Person-25 Person-1
Person-26 Person-27
Person-27 Person-27
Person-28 Person-26
Person-3 Person-3
Person-4 Person-2
Person-6 Person-6
Person-7 Person-7
Person-8 Person-7
Person-9 Person-8


In [11]:
print correct_pred*1.0/test_subjects_count

0.555555555556
